In [94]:
import os, glob, sys, math
import concurrent.futures
import argparse

import shutil

# from scipy.spatial.transform import Rotation as R
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from Bio.PDB import *
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.ResidueDepth import min_dist
from pyquaternion import Quaternion

from utils import pdbtools
from utils import pdb_resdepth
from utils import matrice_distances
from utils import Lennard_Jones
from utils import electrostatic
from utils import combine_methods as cm
from utils import tm_score as tm

# from surface import *
p = PDBParser()

recognized_residues = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET',
                           'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'NH', 'OC']
atom_types = [['N'], ['CA'], ['C'], ['O'], ['GLYCA'],
                  ['ALACB', 'ARGCB', 'ASNCB', 'ASPCB', 'CYSCB', 'GLNCB', 'GLUCB', 'HISCB', 'ILECB', 'LEUCB', 'LYSCB',
                   'METCB', 'PHECB', 'PROCB', 'PROCG', 'PROCD', 'THRCB', 'TRPCB', 'TYRCB', 'VALCB'],
                  ['LYSCE', 'LYSNZ'], ['LYSCD'], ['ASPCG', 'ASPOD1', 'ASPOD2', 'GLUCD', 'GLUOE1', 'GLUOE2'],
                  ['ARGCZ', 'ARGNH1', 'ARGNH2'],
                  ['ASNCG', 'ASNOD1', 'ASNND2', 'GLNCD', 'GLNOE1', 'GLNNE2'], ['ARGCD', 'ARGNE'],
                  ['SERCB', 'SEROG', 'THROG1', 'TYROH'],
                  ['HISCG', 'HISND1', 'HISCD2', 'HISCE1', 'HISNE2', 'TRPNE1'], ['TYRCE1', 'TYRCE2', 'TYRCZ'],
                  ['ARGCG', 'GLNCG', 'GLUCG', 'ILECG1', 'LEUCG', 'LYSCG', 'METCG', 'METSD', 'PHECG', 'PHECD1', 'PHECD2',
                   'PHECE1', 'PHECE2', 'PHECZ', 'THRCG2', 'TRPCG', 'TRPCD1', 'TRPCD2', 'TRPCE2', 'TRPCE3', 'TRPCZ2',
                   'TRPCZ3', 'TRPCH2', 'TYRCG', 'TYRCD1', 'TYRCD2'],
                  ['ILECG2', 'ILECD1', 'ILECD', 'LEUCD1', 'LEUCD2', 'METCE', 'VALCG1', 'VALCG2'], ['CYSSG']]

rng = np.random.default_rng(0)






class SFLA:
    def __init__(self, frogs, mplx_no, n_iter, N, q):
        self.frogs = frogs
        self.mplx_no = mplx_no
        self.FrogsEach = int(self.frogs/self.mplx_no)
        self.weights = [2*(self.FrogsEach+1-j)/(self.FrogsEach*(self.FrogsEach+1)) for j in range(1, self.FrogsEach+1)] 
        self.structinfo = {}
        self.init = 0
        self.mypath ='poses/'
        self.n_iter = n_iter
        self.N = N
        self.q = q
        self.atom_mass = {}
    
    def chaindef(self, file, rec_chain):
        structure = p.get_structure('1bth', file)
        #self.COM[file] = structure.center_of_mass()
        coordinatesr = np.empty((0,3))
        tobi_residuesr = []
        residue_id = []
        boundary_residue_coord = np.empty((0,3))
        atom_coord=np.empty((0,3))
        atom_mass = []
        boundary_residue_id=[]
        boundary_residue_name=[]
        for model in structure:
            surface = get_surface(model)
            for chain in model:
                if chain.id in rec_chain:
                    for residue in chain:
                        cx = 0.0
                        cy = 0.0
                        cz = 0.0
                        count = 0
                        residue_index=recognized_residues.index(residue.get_resname())
                        atom_set=np.empty((0,3))
                        for atom in residue:
                            if  not atom.name=='H':
                                #print(atom.mass)
                                ax=atom.get_coord()[0]
                                ay=atom.get_coord()[1]
                                az=atom.get_coord()[2]
                                atom_set=np.append(atom_set,[atom.get_coord()], axis=0)
                                atom_coord=np.append(atom_coord,[atom.get_coord()], axis=0)
                                atom_mass.append(atom.mass)
                                cur_atom=residue.get_resname()+atom.name
                                for typ in atom_types:
                                    if  cur_atom in typ or atom.name in ['N','CA','C','O']:	#typ:#atom.name now added
                                        cx += ax
                                        cy += ay
                                        cz += az
                                        count += 1
                                    else:
                                        pass
                        cx/= float(count)
                        cy/= float(count)
                        cz/= float(count)
                        coordinatesr=np.append(coordinatesr,[[cx, cy, cz]], axis=0)
                        #rcc+=1
                        tobi_residuesr.append(residue_index)
                        residue_id.append(str(residue.get_id()[1])+residue.get_id()[2])
                        fji=0     #check whether any of of the atoms in the resdue are at a distance 3 A from surface
                        for ji in range(len(atom_set)):
                            if min_dist(atom_set[ji], surface) < 2:
                                fji=1
                                break
                        if fji==1:
                            boundary_residue_coord=np.append(boundary_residue_coord,[[cx, cy, cz]],axis=0)
                            #boundary_atom_name.append(atom.name)
                            boundary_residue_id.append(str(residue.get_id()[1])+residue.get_id()[2])
                            boundary_residue_name.append(residue.get_resname())
        self.atom_mass[file] = np.array(atom_mass)
        return boundary_residue_coord,boundary_residue_name, boundary_residue_id, atom_coord
    
    def findPointNormals(self, points, numNeighbours, viewPoint, residue_id, residue_name, f):
        nbrs = NearestNeighbors(n_neighbors=numNeighbours+1, algorithm='kd_tree').fit(points)
        distances, indices = nbrs.kneighbors(points)
        n = []
        [n.append(indices[i][1:].tolist()) for i in range(0,len(indices))]

        # find difference in position from neighbouring points
        n=np.asarray(n).flatten('F')    
        p = np.tile(points,(numNeighbours,1)) - points[n]
        x=np.zeros((3,len(points),numNeighbours))
        for i in range(0,3):
            for j in range(0,len(points)):
                for k in range(0,numNeighbours):
                    x[i,j,k]=p[k*len(points)+j,i]
        p = x
        C = np.zeros((len(points),6))
        C[:,0]= np.sum(np.multiply(p[0],p[0]),axis=1)
        C[:,1]= np.sum(np.multiply(p[0],p[1]),axis=1)
        C[:,2]= np.sum(np.multiply(p[0],p[2]),axis=1)
        C[:,3]= np.sum(np.multiply(p[1],p[1]),axis=1)
        C[:,4]= np.sum(np.multiply(p[1],p[2]),axis=1)
        C[:,5]= np.sum(np.multiply(p[2],p[2]),axis=1)
        C = np.divide(C, numNeighbours)
        normals = np.zeros((len(points),3))
        curvature = np.zeros((len(points),1))
        for i in range(0,len(points)):
            Cmat = [[C[i,0], C[i,1] ,C[i,2]], [C[i,1], C[i,3], C[i,4]], [C[i,2], C[i,4], C[i,5]]]
            [value,vector] = np.linalg.eigh(Cmat)
            [lam,k] = min(value), value.tolist().index(min(value))
            normals[i,:] = vector[:,k] #np.transpose(vector[:,k])
            curvature[i]= lam / sum(value)

        return normals, curvature

    def find_score(self, args):
        output_file='out' + str(args[1]) + '.pdb'
        pH = 7
        dist = 8.6
        depth = "msms"
        with open(os.path.join(self.mypath, output_file),'w') as out:
            in1 = open(self.rec_filename, "r")
            in2 = open(self.lig_filename, "r")
            for line in in1:
                if "ATOM" in line:
                    out.write(line)
            indexing = 0
            new_co = args[0]
            for line in in2:
                if "ATOM" in line:
                    # print(line)
                    l = line.split()
                    l[0] = l[0].ljust(5)
                    l[1] = l[1].rjust(5)
                    l[2] = l[2].ljust(3)
                    l[3] = l[3].ljust(3)
                    l[4] = line[21]
                    l[5] = ("%4d" % (int(line[22:26]))).rjust(4)
                    l[6] = ("%8.3f" % (float(new_co[indexing][0]))).rjust(8)
                    l[7] = ("%8.3f" % (float(new_co[indexing][1]))).rjust(8)
                    l[8] = ("%8.3f" % (float(new_co[indexing][2]))).rjust(8)
                    out.write(
                        "{0} {1}  {2} {3} {4}{5}    {6}{7}{8}".format(
                            l[0], l[1], l[2], l[3], l[4], l[5], l[6], l[7], l[8]
                        )
                    )
                    out.write("\n")
                    indexing += 1

        pdbfile = os.path.join(self.mypath, output_file)
        my_struct = pdbtools.read_pdb(pdbfile)
        try:
            depth_dict = pdb_resdepth.calculate_resdepth(structure=my_struct, pdb_filename=pdbfile, method=depth)
        except:
            return
        distmat = matrice_distances.calc_distance_matrix(
            structure=my_struct,
            depth=depth_dict,
            chain_R=self.rec_chain,
            chain_L=self.lig_chain,
            dist_max=dist,
            method=depth,
        )

        vdw = Lennard_Jones.lennard_jones(dist_matrix=distmat)
        electro = electrostatic.electrostatic(inter_resid_dict=distmat, pH=pH)
        score = vdw + electro

        return score, args[1], args[2], args[3]

    def pdbpre(self, file1):
        with open(os.path.join(self.path, file1), "r") as pdb_in:   # TODO: Args.pdb correct info
            with open(file1 + "1.pdb", "w") as out: 
                atmno = 1
                resno = 0
                res = ""
                fr = ""
                l = [""] * 11
                for line in pdb_in:
                    if "ATOM" in line[0:4]:
                        li = line.split()
                        l[0] = li[0].ljust(6)
                        l[1] = str(atmno).rjust(4)
                        l[2] = li[2].ljust(3)
                        l[3] = li[3].ljust(3)
                        l[4] = line[21]
                        if fr != line[21]:
                            atmno = 1
                            resno = 0
                            res = ""
                            fr = line[21]
                        if line[22:26] == res:
                            l[5] = ("%4d" % (int(resno))).rjust(4)
                        else:
                            resno += 1
                            res = line[22:26]
                            l[5] = ("%4d" % (int(resno))).rjust(4)
                        # if len(l[6])>10:
                        l[6] = ("%8.3f" % (float(line[29:38]))).rjust(8)
                        l[7] = ("%8.3f" % (float(line[38:46]))).rjust(8)
                        l[8] = ("%8.3f" % (float(line[46:54]))).rjust(8)
                        l[9] = ("%6.2f" % (float(line[55:60]))).rjust(6)
                        l[10] = ("%6.2f" % (float(line[60:66]))).ljust(6)
                        out.write(
                            "{0} {1}  {2} {3} {4}{5}    {6}{7}{8}{9}{10}".format(
                                l[0], l[1], l[2], l[3], l[4], l[5], l[6], l[7], l[8], l[9], l[10]
                            )
                        )
                        out.write("\n")
                        atmno += 1
        return file1 + "1.pdb"

    def generate_one_frog(self, uid):
        Quater = [0, 0, 0, 0]
        recRandIdx = rng.integers(0, self.rec_coord.shape[0] - 1)
        ligRandIdx = rng.integers(0, self.lig_coord.shape[0] - 1)
        axis = self.rec_coord[recRandIdx]
        a = self.rec_normal[recRandIdx]
        b = self.lig_normal[ligRandIdx]

        dotProduct = np.dot(a, b)
        theta = np.arccos(dotProduct) * 2 - np.pi

        Quater = Quaternion(axis=a, angle=theta)

        final = np.array([Quater.rotate(i) for i in self.lig_atom])
        args = [[final, uid, Quater, -1]]
        return args
    
    def generate_one_frog2(self, uid):
        Quater = [0, 0, 0, 0]
        recRandIdx = rng.integers(0, self.rec_coord.shape[0] - 1)
        ligRandIdx = rng.integers(0, self.lig_coord.shape[0] - 1)
        axis = self.rec_coord[recRandIdx]
        a = self.rec_normal[recRandIdx]
        b = self.lig_normal[ligRandIdx]
        tran = a - b
        dotProduct = np.dot(a, b)
        theta = np.arccos(dotProduct) * 2 - np.pi

        Quater = Quaternion(axis=a, angle=theta)

        final = np.array([Quater.rotate(i + tran) for i in self.lig_atom])
        args = [[final, uid, Quater, -1]]
        return args
    
    def generate_one_frog1(self, uid):
        Quater = [0, 0, 0, 0]
        recRandIdx = rng.integers(0, self.rec_coord.shape[0] - 1)
        ligRandIdx = rng.integers(0, self.lig_coord.shape[0] - 1)
        a = self.rec_coord[recRandIdx]
        b = self.lig_coord[ligRandIdx]
        tran = b - a
        dotProduct = np.dot(a, b)
        theta = np.arccos(dotProduct) * 2 - np.pi

        Quater = Quaternion(axis=a, angle=theta)
        #lig_trans = [i + tran for i in self.lig_atom]
        final = np.array([Quater.rotate(i + tran) for i in self.lig_atom])
        args = [[final, uid, Quater, -1]]
        return args
    
    def normalize_vector(self, v):
        """Normalizes a given vector"""
        norm = np.linalg.norm(v)
        if norm < 0.00001:
            return v
        return v / norm


    def quaternion_from_vectors(self, a, b):
        """Calculate quaternion between two vectors a and b.
        Code source: http://lolengine.net/blog/2014/02/24/quaternion-from-two-vectors-final
        """
        u = self.normalize_vector(a)
        v = self.normalize_vector(b)
        norm_u_norm_v = np.sqrt( np.dot(u, u) * np.dot(v, v))
        real_part = norm_u_norm_v + np.dot(u, v)

        if real_part < 1.e-6 * norm_u_norm_v:
            # If u and v are exactly opposite, rotate 180 degrees
            # around an arbitrary orthogonal axis. Axis normalisation
            # can happen later, when we normalise the quaternion.
            real_part = 0.0
            if abs(u[0]) > abs(u[2]):
                w = [-u[1], u[0], 0.]
            else:
                w = [0., -u[2], u[1]]
        else:
            # Otherwise, build quaternion the standard way
            w = np.cross(u, v)

        #return np.quaternion(real_part, w[0], w[1], w[2]).normalized()
        return Quaternion(real_part, w[0], w[1], w[2]).normalised
    
    def quaternion_from_vectors2(self, a, b):
        """Calculate quaternion between two vectors a and b.
        Code source: http://lolengine.net/blog/2014/02/24/quaternion-from-two-vectors-final
        """
        u = self.normalize_vector(a)
        v = self.normalize_vector(b)
        w = np.cross(u, v)
        q = Quaternion(np.dot(u, v), w[0], w[1], w[2])
        q[0] += q.magnitude
        
        return q.normalised

    def translation(self, trans_coord):
        return np.vstack([i + trans_coord for i in self.lig_atom])
    
    def rotation(self, q, lig_atom):
        final = []
        for i in lig_atom:
            v = np.quaternion(0., i[0], i[1], i[2])
            m = np.dot(np.dot(q, v), q.inverse())
            final.append([m.x, m.y, m.z])
        
        return np.array(final)
    
    def tranformation(self, q, trans_coord):
        #lig_new = self.translation(trans_coord)
        #final = self.rotation(q, self.lig_atom)
        final = np.array([q.rotate(i) for i in self.lig_atom])
        return final
    
    def generate_one_frog3(self, uid):
        #Quater = [0, 0, 0, 0]
        recRandIdx = rng.integers(0, self.rec_coord.shape[0] - 1)
        ligRandIdx = rng.integers(0, self.lig_coord.shape[0] - 1)
        a = self.rec_coord[recRandIdx]
        b = self.lig_coord[ligRandIdx]
        quater = self.quaternion_from_vectors(a, b)
        #lig_trans = a - b
        lig_trans = self.normalize_vector(a) - self.normalize_vector(b)
        #u = self.normalize_vector(a)
        #v = self.normalize_vector(b)
        final = self.tranformation(quater, lig_trans)
        args = [[final, uid, quater, lig_trans, -1]]
        return args 
    
    def generate_init_population1(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            Doargs = []
            for _ in range(self.frogs):
                Doargs += self.generate_one_frog1(self.init)
                self.init += 1
            
            results = executor.map(self.find_score, Doargs)
            for r in results:
                if r:
                    self.structinfo[r[1]] = [r[0], r[2], r[3]]      
    
    def generate_init_population(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            Doargs = []
            for _ in range(self.frogs):
                Doargs += self.generate_one_frog2(self.init)
                self.init += 1
            
            results = executor.map(self.find_score, Doargs)
            for r in results:
                if r:
                    self.structinfo[r[1]] = [r[0], r[2]]   

    def sort_frog(self):
        sorted_fitness = np.array(sorted(self.structinfo, key = lambda x: self.structinfo[x][0]))

        memeplexes = np.zeros((self.mplx_no, self.FrogsEach))

        for j in range(self.FrogsEach):
            for i in range(self.mplx_no):
                memeplexes[i, j] = sorted_fitness[i + (self.mplx_no*j)] 
        return memeplexes


    
    def local_search_one_memeplex(self, im):
        """
            q: The number of frogs in submemeplex
            N: No of mutations
        """

        for iN in range(self.N):
            uId = self.init + im + 1
            rValue = rng.random(self.FrogsEach) * self.weights                      # random value with probability weights
            subindex = np.sort(np.argsort(rValue)[::-1][0:self.q])                  # index of selected frogs in memeplex
            submemeplex = self.memeplexes[im][subindex] 

            #--- Improve the worst frog's position ---#
            # Learn from local best Pb #
            Pb = self.structinfo[int(submemeplex[0])]                               # mark the best frog in submemeplex
            Pw = self.structinfo[int(submemeplex[self.q - 1])]                      # mark the worst frog in submemeplex

            S = rng.random() * (Pb[1] - Pw[1]) 
            Uq = Pw[1] + S

            globStep = False
            censorship = False
            
            # Check feasible space and the performance #
            if self.omega[0] <= min(Uq) and max(Uq) <= self.omega[1]:
                final = np.array([Uq.rotate(i) for i in self.lig_atom])  
                results = self.find_score([final, uId, Uq, im])

                if results[0] > Pw[0]:
                    globStep = True
            else: 
                globStep = True

            if globStep:
                S = rng.random() * (self.Frog_gb[1] - Pw[1])
                for i in range(4):
                    if S[i] > 0:
                        S[i] = min(S[i], self.max_step)
                    else:
                        S[i] = max(S[i], -self.max_step)
                Uq = Pw[1] + S

                if self.omega[0] <= min(Uq) and max(Uq) <= self.omega[1]:
                    final = np.array([Uq.rotate(i) for i in self.lig_atom])  
                    results = self.find_score([final, uId, Uq, im])
                    if results[0] > Pw[0]:
                        censorship = True
                else:
                    censorship = True

            if censorship:
                params = self.generate_one_frog(uId)
                results = self.find_score(params)            


            #StructInfo[im] = [results[0], results[2]]
            shutil.move(os.path.join('poses/', 'out'+str(uId)+'.pdb'), os.path.join('poses/', 'out'+ str(submemeplex[self.q-1]) + '.pdb'))
            self.structinfo[int(submemeplex[self.q-1])] = [results[0], results[2]]
            self.memeplexes[im] = self.memeplexes[im][np.argsort(self.memeplexes[im])]
            
    def local_search(self):
        self.Frog_gb = self.structinfo[int(self.memeplexes[0][0])]
    
        with concurrent.futures.ProcessPoolExecutor() as executor:
            doargs = [[im] for im in range(len(self.memeplexes))]
            results = executor.map(self.local_search_one_memeplex, doargs)
    
    def shuffle_memeplexes(self):
        """Shuffles the memeplexes and sorting them.
        """
        temp = self.memeplexes.flatten()
        temp = np.array(sorted(temp, key = lambda x: self.structinfo[x][0]))
        for j in range(self.FrogsEach):
            for i in range(self.mplx_no):
                self.memeplexes[i, j] = temp[i + (self.mplx_no * j)]    
            
    def run_sfla(self, data_path, protein_name, rec_name, lig_name):
        self.path = data_path
        self.rec_name = rec_name
        self.lig_name = lig_name
        
        rpdb = rec_name + '_model_st.pdb'
        lpdb = lig_name + '_model_st.pdb'
        
        self.rec_chain = [i for i in rec_name]
        self.lig_chain = [i for i in lig_name]
         
        self.rec_filename = self.pdbpre(rpdb) # INP2
        self.lig_filename = self.pdbpre(lpdb) # INP1
        
        self.rec_coord, rec_res, rec_res_id, self.rec_atom = self.chaindef(self.rec_filename, self.rec_chain)   
        self.lig_coord, lig_res, lig_res_id, self.lig_atom = self.chaindef(self.lig_filename, self.lig_chain)
        
        self.rec_normal, rec_curve = self.findPointNormals(self.rec_coord, 20,[0,0,0], rec_res_id, rec_res, 'r')
        self.lig_normal, lig_curve = self.findPointNormals(self.lig_coord, 20,[0,0,0], lig_res_id, lig_res, 'r')
        
        self.generate_init_population()
        self.memeplexes = self.sort_frog(self.mplx_no)
        
        self.omega = [np.amin(self.rec_normal), np.amax(self.rec_normal)]
        self.max_step = (self.omega[1] - self.omega[0])/2
        
        for _ in range(self.n_iter):
            self.local_search()
            self.shuffle_memeplexes()
        
        directory = "native_" + protein_name
        final_path = os.path.join("./", directory)
        os.mkdir(final_path)
        # move best global frog to native folder
        shutil.move(os.path.join("poses/", "out" + str(self.memeplexes[0][0]) + ".pdb"), directory)
    
    def run_sfla1(self, data_path, protein_name, rec_name, lig_name):
        
        self.path = data_path
        self.rec_name = rec_name
        self.lig_name = lig_name
        
        rpdb = rec_name + '_model_st.pdb'
        lpdb = lig_name + '_model_st.pdb'
        
        self.rec_chain = [i for i in rec_name]
        self.lig_chain = [i for i in lig_name]
         
        self.rec_filename = self.pdbpre(rpdb) # INP2
        self.lig_filename = self.pdbpre(lpdb) # INP1
        
        self.rec_coord, rec_res, rec_res_id, self.rec_atom = self.chaindef(self.rec_filename, self.rec_chain)   
        self.lig_coord, lig_res, lig_res_id, self.lig_atom = self.chaindef(self.lig_filename, self.lig_chain)
        
        self.rec_normal, rec_curve = self.findPointNormals(self.rec_coord, 20,[0,0,0], rec_res_id, rec_res, 'r')
        self.lig_normal, lig_curve = self.findPointNormals(self.lig_coord, 20,[0,0,0], lig_res_id, lig_res, 'r')
        
        self.generate_init_population()

In [9]:
slfa = SFLA(50, 10, 6, 10, 4)
slfa.run_sfla1("Data/4dn4_LH:M/", "4dn4", "LH", "M")

MSMS running for poses/out0.pdb
MSMS running forposes/out4.pdbMSMS running for  poses/out3.pdb

MSMS running forMSMS running for  poses/out5.pdbposes/out1.pdb

MSMS running for poses/out7.pdbMSMS running for
 poses/out6.pdb
MSMS running forMSMS running for  poses/out9.pdbposes/out2.pdb

MSMS running for poses/out8.pdbMSMS running for
 poses/out10.pdb
MSMS running for poses/out11.pdb
MSMS running for poses/out13.pdb
MSMS running forposes/out14.pdb 
MSMS running forposes/out15.pdb 
MSMS running for
 poses/out16.pdbMSMS running for poses/out12.pdb
MSMS running for poses/out18.pdb
MSMS running for poses/out17.pdb
MSMS running for poses/out20.pdbMSMS running for
 MSMS running forMSMS running forposes/out19.pdb  poses/out22.pdb

MSMS running forposes/out21.pdb 
poses/out23.pdb
MSMS running forMSMS running for MSMS running forposes/out25.pdb 
MSMS running forposes/out24.pdb 
MSMS running for poses/out29.pdb
 poses/out27.pdb
MSMS running for poses/out26.pdb
MSMS running for poses/out28.pdbpose

In [14]:
quat = slfa.structinfo[1][1]

In [17]:
quat.magnitude

1.0

In [18]:
quat1 = slfa.structinfo[2][1]

In [19]:
quat1.magnitude

1.0

In [20]:
quat1.angle

0.05876807940738127

In [21]:
lig_atom = slfa.lig_atom
rec_coord = slfa.rec_coord
lig_coord = slfa.lig_coord
lig_normal = slfa.lig_normal
rec_normal = slfa.rec_normal

In [22]:
rec_coord[0]

array([-34.67484472,  41.98275331,  -2.3416753 ])

In [23]:
lig_coord[0]

array([-72.93397422,  70.44146086,   9.80980285])

In [37]:
quat3 = sfla.quaternion_from_vectors(rec_coord[0], lig_coord[5])

In [30]:
quat4 = sfla.quaternion_from_vectors2(rec_coord[0], lig_coord[0])

In [41]:
quat4.angle

0.1790243137780756

In [31]:
quat3.axis

array([0.5834133 , 0.51680504, 0.62653131])

In [34]:
quat1.axis

array([-0.0628678 , -0.11303486,  0.9916001 ])

In [35]:
quat.axis

array([ 0.22902477,  0.06642679, -0.97115145])

In [39]:
quat3.angle

0.06383115626955949

In [38]:
quat3

Quaternion(0.9994907411661257, 0.01965713098685071, 0.017254976312409492, 0.018278985666959766)

In [25]:
sfla = SFLA(50, 10, 6, 10, 4)
sfla.run_sfla1("Data/4dn4_LH:M/", "4dn4", "LH", "M")

MSMS running for poses/out0.pdb
MSMS running forMSMS running forMSMS running for   MSMS running forposes/out7.pdbMSMS running forposes/out6.pdbMSMS running forposes/out4.pdb 

  
poses/out1.pdbposes/out5.pdbposes/out2.pdb
MSMS running for

 poses/out3.pdb
MSMS running for poses/out10.pdb
MSMS running for poses/out9.pdb
MSMS running for poses/out11.pdb
MSMS running for poses/out8.pdb
MSMS running forMSMS running for  poses/out14.pdbposes/out13.pdb

MSMS running for  poses/out15.pdbMSMS running for
poses/out12.pdb
MSMS running for poses/out16.pdb
MSMS running for poses/out17.pdb
MSMS running for poses/out18.pdb
MSMS running forMSMS running for  poses/out20.pdbposes/out19.pdb

MSMS running for poses/out22.pdb
MSMS running for MSMS running for poses/out21.pdbposes/out23.pdb
MSMS running for
 poses/out25.pdb
MSMS running for poses/out24.pdb
MSMS running forMSMS running for MSMS running for MSMS running for  poses/out28.pdbposes/out26.pdb
poses/out27.pdb


MSMS running forposes/out29.pdb pos

In [45]:
def center_of_coordinates(atoms):
    """Calculates the center of coordinates"""
    dimension = len(atoms)
    total_x = 0.
    total_y = 0.
    total_z = 0.
    if dimension:
        for atom in atoms:
            total_x += atom[0]
            total_y += atom[1]
            total_z += atom[2]
        return [total_x/dimension, total_y/dimension, total_z/dimension]
    else:
        return [0., 0., 0.]

array([-74.54000092,  71.06300354,   9.52200031])

In [46]:
cen = center_of_coordinates(lig_atom)

In [66]:
cen2 = center_of_coordinates(sfla.rec_atom)

In [67]:
cen2

[-21.52373253220308, 42.7133796800699, -28.35688401056579]

In [47]:
cen

[-58.62122713008397, 56.94977990674301, -3.68469717798748]

In [51]:
def move_to_origin(atoms, center):
    """Moves the structure to the origin of coordinates"""
    translation = [-1*c for c in center]
    change = [atom + translation for atom in atoms]
    return np.vstack(change)

In [73]:
def move_back(atoms, center):
    """Moves the structure to the origin of coordinates"""
    change = [atom + center for atom in atoms]
    return np.vstack(change)

In [52]:
chan = move_to_origin(lig_atom, cen)

In [77]:
chan3 = move_back(final, cen)

In [75]:
lig_atom

array([[-74.54000092,  71.06300354,   9.52200031],
       [-73.55699921,  69.94200134,   9.6260004 ],
       [-72.14900208,  70.53199768,   9.7130003 ],
       ...,
       [-55.79600143,  41.76200104,   4.64099979],
       [-56.02799988,  42.14599991,   3.4920001 ],
       [-54.8370018 ,  40.88199997,   4.921     ]])

In [78]:
chan3

array([[-74.58620277,  67.74130726,  12.30498039],
       [-73.48372167,  66.73757631,  12.2009905 ],
       [-72.14503396,  67.47594335,  12.23014383],
       ...,
       [-53.38970134,  41.44686001,   2.6602512 ],
       [-53.77269647,  41.91278127,   1.5842815 ],
       [-52.31947054,  40.6611402 ,   2.75890569]])

In [68]:
chan2 = move_to_origin(sfla.rec_atom, cen2)

In [57]:
final = np.array([quat4.rotate(i) for i in chan])

In [59]:
sfla.find_score([final, 100, quat4, -1])

MSMS running for poses/out100.pdb
MSMS finished with poses/out100.pdb


(0,
 100,
 Quaternion(0.9959964611274629, 0.05215287226792545, 0.046198582047412874, 0.05600730624288485),
 -1)

In [60]:
sfla.find_score([chan, 101, quat4, -1])

MSMS running for poses/out101.pdb
MSMS finished with poses/out101.pdb


(0,
 101,
 Quaternion(0.9959964611274629, 0.05215287226792545, 0.046198582047412874, 0.05600730624288485),
 -1)

In [63]:
quat1

Quaternion(0.9995683201668347, -0.0018470439618483033, -0.003320942868596332, 0.0291330239748412)

In [64]:
quat3

Quaternion(0.9994907411661257, 0.01965713098685071, 0.017254976312409492, 0.018278985666959766)

In [83]:
quat3.angle

0.06383115626955949

In [84]:
quat3.axis

array([0.6160148 , 0.54073612, 0.57282651])

In [82]:
sfla.find_score([chan3, 102, quat4, -1])

MSMS running for poses/out102.pdb
MSMS finished with poses/out102.pdb


(602.0637148316767,
 102,
 Quaternion(0.9959964611274629, 0.05215287226792545, 0.046198582047412874, 0.05600730624288485),
 -1)

In [80]:
final1 = np.array([quat4.rotate(i) for i in lig_atom])

In [85]:
sfla.find_score([final1, 103, quat4, -1])

MSMS running for poses/out103.pdb
MSMS finished with poses/out103.pdb


(0,
 103,
 Quaternion(0.9959964611274629, 0.05215287226792545, 0.046198582047412874, 0.05600730624288485),
 -1)

In [ ]:
sfla1 = SFLA(50, 10, 6, 10, 4)
sfla1.run_sfla1("Data/4dn4_LH:M/", "4dn4", "LH", "M")

In [104]:
np.zeros(3)

array([0., 0., 0.])

In [110]:
def find_center_of_mass(file): 
    c = np.zeros(3)
    total_mass = np.sum(sfla1.atom_mass['M_model_st.pdb1.pdb'])
    for idx, i in enumerate(sfla1.lig_atom):
        c += i * sfla1.atom_mass['M_model_st.pdb1.pdb'][idx]
    com = c/total_mass
    return com

In [111]:
c/total_mass

array([-58.71657658,  57.0304519 ,  -3.67327578])

In [ ]:
sfla1.atom_mass['M_model_st.pdb1.pdb']

In [114]:
np.sum(sfla1.atom_mass['M_model_st.pdb1.pdb'])

7560.8202

In [115]:
total_mass

7560.820199999934

In [117]:
sfla1.atom_mass['M_model_st.pdb1.pdb'].shape

(568,)

In [133]:
np.divide(np.sum((sfla1.lig_atom.T * sfla1.atom_mass['M_model_st.pdb1.pdb']).T, axis=0), total_mass)

array([-58.71657658,  57.0304519 ,  -3.67327578])

In [128]:
c

array([-443945.47827709,  431196.99270862,  -27772.97772868])

In [ ]:
(sfla1.lig_atom.T * sfla1.atom_mass['M_model_st.pdb1.pdb']).T

In [ ]:
def find_center_of_mass(file):
    c = (sfla1.lig_atom.T * sfla1.atom_mass['M_model_st.pdb1.pdb']).T
    total_mass = np.sum(sfla1.atom_mass['M_model_st.pdb1.pdb'])
    com = np.divide(np.sum(c, axis=0), total_mass)
    return com

In [ ]:
def translation(self, atom, trans_coord):
    return np.vstack([i + trans_coord for i in atom])

def rotation(self,atom, q):
    atm_org = move_to_origin(atom, com)
    atm_rot = np.array([q.rotate(i) for i in atm_org])
    final = move_back(atm_rot, com)
    return final

def tranformation(self, q, trans_coord):
    lig_new = self.rotation(q, self.lig_atom)
    final = self.translation(lig_new, trans_coord)
    return final

In [134]:
def translation(lig_atom, trans_coord):
    return np.vstack([i + trans_coord for i in lig_atom])

def rotation(q, lig_atom):
    atm_org = move_to_origin(lig_atom, com)
    atm_rot = np.array([q.rotate(i) for i in atm_org])
    final = move_back(atm_rot, com)
    return final

def tranformation(self, q, trans_coord):
    lig_new = self.rotation(q, lig_atom)
    final = self.translation(lig_new, trans_coord)
    return final

In [179]:
rec_coord[0] - lig_coord[0]

array([ 38.2591295 , -28.45870755, -12.15147815])

In [221]:
trans_cor = rng.random(3)

In [223]:
trans_cor = rng.uniform(-20, 20, 3)

In [224]:
trans_cor

array([ -5.39559327, -15.78018882,   5.16432606])

In [171]:
tran_cor = np.clip(rec_coord[0] - lig_coord[0], -20, 20)

In [160]:
rec_coord[0]

array([-34.67484472,  41.98275331,  -2.3416753 ])

In [151]:
np.max(rec_coord, axis=0)

array([11.60560268, 72.50960726, -1.67627271])

In [152]:
np.min(rec_coord, axis=0)

array([-50.32043392,  15.21637826, -60.5767498 ])

In [150]:
np.min(lig_coord, axis=0)

array([-72.93397422,  43.72577875, -15.1023423 ])

In [ ]:
np.max()

In [215]:
i

array([-54.8370018 ,  40.88199997,   4.921     ])

In [214]:
lig_coord[0]

array([-72.93397422,  70.44146086,   9.80980285])

In [226]:
trans_cor

array([ -5.39559327, -15.78018882,   5.16432606])

In [225]:
lig_coord[0] + trans_cor

array([-78.32956749,  54.66127204,  14.97412891])

In [217]:
k = lig_coord[0] 

In [220]:
tran_cor

array([[0.05856803, 0.33611706, 0.15027947]])

In [219]:
np.add(i, tran_cor)

array([[-54.77843377,  41.21811703,   5.07127947]])

In [145]:
com

array([-58.71657658,  57.0304519 ,  -3.67327578])

In [136]:
quat3

Quaternion(0.9994907411661257, 0.01965713098685071, 0.017254976312409492, 0.018278985666959766)

In [137]:
quat4

Quaternion(0.9959964611274629, 0.05215287226792545, 0.046198582047412874, 0.05600730624288485)

In [227]:
final2 = translation(lig_atom, trans_cor)

In [228]:
final2

array([[-79.93559419,  55.28281472,  14.68632637],
       [-78.95259248,  54.16181253,  14.79032647],
       [-77.54459535,  54.75180886,  14.87732636],
       ...,
       [-61.1915947 ,  25.98181222,   9.80532586],
       [-61.42359315,  26.36581109,   8.65632616],
       [-60.23259507,  25.10181115,  10.08532607]])

In [232]:
org_atom = move_to_origin(lig_atom, com)

In [235]:
com1 = np.array([com[0], -com[1], com[2]])

In [236]:
com1 

array([-58.71657658, -57.0304519 ,  -3.67327578])

In [239]:
final2 = move_back(org_atom, com1)

In [ ]:
org_atom

In [240]:
sfla.find_score([final2, 104, quat4, -1])

MSMS running for poses/out104.pdb
MSMS finished with poses/out104.pdb


(0,
 104,
 Quaternion(0.9959964611274629, 0.05215287226792545, 0.046198582047412874, 0.05600730624288485),
 -1)

In [247]:
def find_center_of_mass1(file):
    c = (sfla1.rec_atom.T * sfla1.atom_mass['LH_model_st.pdb1.pdb']).T
    total_mass = np.sum(sfla1.atom_mass['LH_model_st.pdb1.pdb'])
    com = np.divide(np.sum(c, axis=0), total_mass)
    return com

In [ ]:
sfla1.atom_mass['LH_model_st.pdb1.pdb']

In [251]:
org_rec = find_center_of_mass1("")

In [252]:
org_rec = move_to_origin(sfla1.rec_atom, com1)

In [253]:
org_rec

array([[-11.88381361,   0.47861556,  25.4897044 ],
       [-12.91181349,  -0.19038316,  26.34970429],
       [-13.66281294,  -1.29938432,  25.61570432],
       ...,
       [ 15.42218805,  -1.26138231, -26.9132962 ],
       [ 14.43318773,  -0.34738466, -27.61229751],
       [ 14.60818792,   1.05561522, -27.08629653]])

In [254]:
def get_minmax_crd(rec_coord, fname_times = 2, fname_buff = 0.0):
    ''' 
    Obtain the min max coordinates from a receptors conformational space to generate sensible structures for ensembles in JabberDock.
    We move the ligand around this space to represent the size the receptor occupies, but we do want it to be slightly bigger.
    :param fname_rec: PDB file name of receptor
    :param fname_times: Number greater than 1 to represent extra space added to search space (default is 2, so twice as big as the receptor)
    :param fname_buff: Buffer region at the edges to include (default is 0.0 ang.)
    :returns: numpy array size 3, containing the x, y and z lengths that provide the sample space about which the ligand will move in.
    '''
 
    rec_minx = np.min(rec_coord[:, 0])
    rec_miny = np.min(rec_coord[:, 1])
    rec_minz = np.min(rec_coord[:, 2])

    rec_maxx = np.max(rec_coord[:, 0])
    rec_maxy = np.max(rec_coord[:, 1])
    rec_maxz = np.max(rec_coord[:, 2])

    rec_x = rec_maxx - rec_minx
    rec_y = rec_maxy - rec_miny
    rec_z = rec_maxz - rec_minz

    rec_x *= fname_times
    rec_y *= fname_times
    rec_z *= fname_times

    rec_x += fname_buff
    rec_y += fname_buff
    rec_z += fname_buff

    return np.array((rec_x, rec_y, rec_z))

In [ ]:
def get_minmax_crd(times = 2):
    ''' 
    Obtain the min max coordinates from a receptors conformational space to generate sensible structures for ensembles.
    We move the ligand around this space to represent the size the receptor occupies, but we do want it to be slightly bigger.
    :param fname_times: Number greater than 1 to represent extra space added to search space (default is 2, so twice as big as the receptor)
    :returns: numpy array size 3, containing the x, y and z lengths that provide the sample space about which the ligand will move in.
    '''

    coord = (np.max(self.receptor.coord, axis=0) - np.min(self.receptor.coord, axis=0)) * times 
    return coord

In [255]:
get_minmax_crd(org_rec)

array([138.83999825, 120.01399612, 128.63599682])

In [ ]:
org_rec

array([123.8520732 , 114.586458  , 117.80095419])

In [ ]:
def new_step(q1, t1, q2, t2):
    q_new = Quaternion.slerp(q1, q2, rng.random())
    coord_new = (t2 - t1)* rng.random()
    return q_new, coord_new